In [12]:
import pyspark # run after findspark.init() if you need it
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, split
spark.sparkContext.stop()
# Start up your pyspark session as always
spark = SparkSession.builder.appName("TwitterStream").getOrCreate()
spark

# read the tweet data from socket
tweet_df = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 60439) \
    .load()

# type cast the column value
tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING)")

# Then split words based on space, filter out only hashtag (#) values and group them up.
tweets_tab = tweet_df_string.withColumn('word', explode(split(col('value'), ' '))) \
    .groupBy('word') \
    .count() \
    .sort('count', ascending=False). \
    filter(col('word').contains('#'))

writeTweet = tweets_tab \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("tweetquery") \
    .start()
print("----- streaming is running -------")



23/01/19 23:11:05 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
23/01/19 23:11:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/dn/6362h3w12mqfb_jh4vg90vp40000gn/T/temporary-e81f60e4-d28e-476b-b80d-73f39b8353da. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/19 23:11:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
----- streaming is running -------


23/01/19 23:11:05 ERROR MicroBatchExecution: Query tweetquery [id = 224a57b7-1bd5-4a00-bcae-499c76637387, runId = 169a7ea9-0f67-4ce5-9015-03388e1ca286] terminated with error
java.net.ConnectException: Connection refused (Connection refused)
	at java.base/java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:412)
	at java.base/java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:255)
	at java.base/java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:237)
	at java.base/java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.base/java.net.Socket.connect(Socket.java:615)
	at java.base/java.net.Socket.connect(Socket.java:563)
	at java.base/java.net.Socket.<init>(Socket.java:459)
	at java.base/java.net.Socket.<init>(Socket.java:236)
	at org.apache.spark.sql.execution.streaming.sources.TextSocketMicroBatchStream.initialize(TextSocketMicroBatchStream.scal